# EXPLAINATION
this is a small example showing how to integrate all of this into a workflow to simulate gyroids.
This workflow expects a folder tree as follow
- /main folder
    - /01 - STL files
    - /02 - mesh files
    - /03 - inp files
        - generate_sim.py (you will need to write this script yourself)(there is an example for one in the examples folder)
    - /04 - odb files 
- full_dimulation_workflow.py (this file)


In [ ]:
import numpy as np
from stl import mesh as stl_mesh
import plotly.graph_objects as go  # for visualization
import os
import open3d as o3d
import trimesh
from pathlib import Path
import meshio
import subprocess
import time
import shutil

import gyroid_utils

working_path = os.getcwd()
print("Current working directory:", working_path)
STL_path = os.path.join(working_path, '01 - STL files\\')
print("STL path:", STL_path)
MESH_path = os.path.join(working_path, '02 - mesh files\\')
print("MESH path:", MESH_path)
INP_path = os.path.join(working_path, '03 - inp files\\')
print("INP path:", INP_path)
ODB_path = os.path.join(working_path, '04 - odb files\\')
print("ODB path:", ODB_path)




#==============================
#======= define functions =========
#==============================

In [ ]:
def process_single_gyroid(config:dict, 
                          size_x, 
                          size_y, 
                          size_z, 
                          dx_grid, 
                          dy_grid, 
                          dz_grid,
                          mesh_size:int = 1000000):
    """
    Process a single gyroid configuration.
    Parameters:
        config (dict): Configuration dictionary with keys:
            file_name, sigma_x, sigma_y, sigma_z, alpha, beta, gamma, t, w_T, w_II
        size_x, size_y, size_z: domain sizes
        dx_grid, dy_grid, dz_grid: grid spacing
    """
    # Reconstruct grids locally in this worker process
    x1 = np.arange(-size_x/2, size_x/2+dx_grid, dx_grid)
    y1 = np.arange(-size_y/2, size_y/2+dy_grid, dy_grid)
    z1 = np.arange(-size_z/2, size_z/2+dz_grid, dz_grid)
    x, y, z = np.meshgrid(x1, y1, z1, indexing='ij')

    
    file_name = config['file_name']
    t = config['t']
    w_T = config['w_T']
    w_II = config['w_II']
    
    px = np.zeros_like(x) + w_T 
    py = px
    pz = np.zeros_like(z) + w_II
    t = np.zeros_like(x) + t 

    # ==== step 1 ========
    gyroid_utils.set_log_level(level="WARNING")  # suppress detailed logs for parallel runs
    # Time each step
    t0 = time.time()    
    
    is_gyroid_succesfully_defined = gyroid_utils.gyroid.create_a_gyroid(x, y, z, px, py, pz, t, STL_path + file_name, 
                    step_size=2, simplification_factor=1500000, field_mode="distance")
    if not is_gyroid_succesfully_defined:
        if mesh_size < 3000000:  # if we haven't already tried with a finer mesh
            print(f"[{file_name}] Gyroid generation failed due to invalid mesh quality. Trying with a finer mesh.")
            process_single_gyroid(config, size_x, size_y, size_z,  dx_grid*1.2,  dy_grid*1.2, dz_grid*1.2, mesh_size*1.5)
        else:
            print(f"[{file_name}] Gyroid generation failed due to invalid mesh quality. Reached max mesh size.Skipping this configuration.")
        return
    
    t1 = time.time()
    print(f"[{file_name}] Gyroid created: {t1 - t0:.2f}s")
    del x, y, z, amp, px, py, pz, t  # free memory
    
    # ==== step 2 ========
    gyroid_utils.TET_mesh_tools.mesh_an_STL(STL_path, MESH_path, file_name, 
                                            stop_energy = 30.0, 
                                            epsilon= 0.002, 
                                            CPU_cores=2)
    
    t2 = time.time()
    print(f"[{file_name}] Mesh created: {t2 - t1:.2f}s")
    
    # ==== step 3 ========
    gyroid_utils.abaqus_tools.create_simulation(input_path=MESH_path, 
                      output_path=INP_path, 
                      file_name=file_name, 
                      script_name = "generate_frequency_sim.py")
    t3 = time.time()
    print(f"[{file_name}] Simulation created: {t3 - t2:.2f}s")
    
    # ==== step 4 ========
    gyroid_utils.abaqus_tools.run_simulation(input_path=INP_path, 
                   output_path=ODB_path, 
                   file_name=file_name)
    t4 = time.time()
    print(f"[{file_name}] Simulation started: {t4 - t3:.2f}s")
    # ==== step 5 ========
    gyroid_utils.abaqus_tools.wait_for_simulation_completed(ODB_path=ODB_path, file_name=file_name)
    t5 = time.time()
    print(f"[{file_name}] Simulation completed: {t5 - t4:.2f}s")
    print(f"[{file_name}] TOTAL TIME: {t5 - t0:.2f}s")
    
    return file_name, t5 - t0
    


#=========================================
#======= test of the 3d gaussian =========
#=========================================

In [ ]:
# size of domain
size_x = 300
size_y = 300
size_z = 300

# --- Discretization of the domain ---
# Resolution in each axis, calculated as a funcion of the size of the gyroid's unit cell
dx_grid = size_x / 450
dy_grid = size_y / 450
dz_grid = size_z / 450


# ==============================================
# ============ generate gyroids =================
# ==============================================
# Define your  gyroid configurations
gyroid_configs = [
    {   'file_name' : "test", 
        't': 6,
        'w_T': 40.0,
        'w_II': 40.0,
    }]


num_configs = 5000


# ========== Latin Hypercube Sampling  ==========
from scipy.stats import qmc

# Define parameter bounds
bounds = [
    [5, 20],      # t
    [40, 100],       # w_T
    [40, 100],       # w_II
]

# Generate Latin Hypercube samples
sampler = qmc.LatinHypercube(d=3, seed=42)  # 6 parameters, reproducible with seed
lhs_samples = sampler.random(n=num_configs) 

# Scale samples to parameter bounds

scaled_samples = qmc.scale(lhs_samples, [b[0] for b in bounds[1:]], [b[1] for b in bounds[1:]])

# Create configurations from LHS samples
for I in range(num_configs):
    t = scaled_samples[I, 0]
    w_T = scaled_samples[I, 1] 
    w_II = scaled_samples[I, 2] 
    file_name = f"config_{I}"
    gyroid_configs.append({
        'file_name': file_name,
        't': t,
        'w_T': w_T,
        'w_II': w_II,
    })
    print(f"Generated configuration {file_name} with t={t:.1f}, w_T={w_T:.1f}, w_II={w_II:.1f}")

#save the configurations for later reference as a csv file
import pandas as pd
df = pd.DataFrame(gyroid_configs)
df.to_csv(STL_path + "gyroid_configs.csv", index=False)
print(f"Generated {len(gyroid_configs)} gyroid configurations and saved to CSV here: {STL_path}gyroid_configs.csv")


# Create partial function with grid parameters already bound
process_single_gyroid(config=gyroid_configs[0],
                          size_x=size_x,
                          size_y=size_y,
                          size_z=size_z,
                          dx_grid=dx_grid,
                          dy_grid=dy_grid,
                          dz_grid=dz_grid)
